In [7]:
import scripts.IO_utils as io
import scripts.NN as NN

In [8]:
import os
import sys
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import random

In [9]:
path = os.path.join(os.path.sep, 'Users', 'egilbertson', 'Box', 'UCSF','Winter2020','Algorithms','Final_Project_Skeleton','data')

In [10]:
pos_seqs = io.read_positives()
neg_seqs = io.read_negatives(pos_seqs)
pos_seqs = io.to_binary(pos_seqs)
neg_seqs = io.to_binary(neg_seqs)
master_seqs = list(io.master_training_set(pos_seqs, neg_seqs))
master_seqs = io.string_to_array(master_seqs)

In [11]:
[0,1,2,3][3:]

[3]

In [12]:
nn = NN.NeuralNetwork(master_seqs, output_dim = 2, reg_term = 0, rounds=100)
err = nn.fit()
nn.l2.round(0) 



ValueError: shapes (13837,102) and (104,3) not aligned: 102 (dim 1) != 104 (dim 0)

In [ ]:
mse = np.average(np.square(nn.backprop()))
mse

In [ ]:
p = nn.l2

In [ ]:
 p = p.round(0)

In [ ]:
p[0][0] ==1

In [ ]:
sum = 0
for i in p:
    if i[0] == 0:
        sum += 1
        
sum

In [ ]:
mse = np.average(np.square(nn.backprop()))
mse